In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

In [ ]:
using LinearAlgebra
using PyPlot
using SparseArrays
using ControlSystems
using ForwardDiff

In [ ]:
# Discrete dynamics
h = 0.1   # time step
A = [1 h; 0 1]
B = [0.5*h*h; h]
C = [1.0 0]

In [ ]:
# Noise Covariances
W = B*0.1*B' #Corresponds to white noise force input to dynamics
V = 0.1 #Noise on position measurements

In [ ]:
n = 2     # number of state
m = 1     # number of controls
Tfinal = 10.0 # final time
N = Int(Tfinal/h)+1    # number of time steps
thist = Array(range(0,h*(N-1), step=h));

In [ ]:
# LQR Cost weights
Q = Array(1.0*I(n))
R = Array(0.1*I(m))
Qn = Q

In [ ]:
#Cost function
function J(xhist,uhist)
    cost = 0.5*xhist[:,end]'*Qn*xhist[:,end]
    for k = 1:(size(xhist,2)-1)
        cost = cost + 0.5*xhist[:,k]'*Q*xhist[:,k] + 0.5*(uhist[k]'*R*uhist[k])[1]
    end
    return cost
end

In [ ]:
#Infinite-Horizon LQR Gain and cost-to-go
P = dare(A,B,Q,R)
K = dlqr(A,B,Q,R)

In [ ]:
# Initial conditions
x0 = [1.0; 0] #True state

#Filter state
Σ0 = Array(1.0*I(2))
x̂0 = [0.0; 0.0]

In [ ]:
xhist = zeros(n,N)
xhist[:,1] .= x0;

uhist = zeros(N)
yhist = zeros(N)
x̂hist = zeros(n,N)
Σhist = zeros(n,n,N);

In [ ]:
#Initial Time Step

#Generate Measurement
yhist[1] = (C*xhist[:,1])[1] + sqrt(V)*randn()

z = yhist[1] - (C*x̂0)[1] #Innovation
S = (C*Σ0*C')[1] + V #Innovation Covariance

L = Σ0*C'*inv(S) #Kalman Gain

x̂hist[:,1] = x̂0 + L*z
Σhist[:,:,1] .= (I-L*C)*Σ0*(I-L*C)' + L*V*L'

uhist[1] = -(K*x̂hist[:,1])[1] #Control input

xhist[:,2] .= A*xhist[:,1] + B*uhist[1] + sqrt(W)*randn(n) #Simulate with stochastic dynamics

In [ ]:
for k = 2:(N-1)
    
    #Generate measurement
    yhist[k] = (C*xhist[:,k])[1] + sqrt(V)*randn()

    #KF Update
    x̃ = A*x̂hist[:,k-1] + B*uhist[k-1] #State Prediction
    Σ̃ = A*Σhist[:,:,k-1]*A' + W #Covariance Prediction

    z = yhist[k] - (C*x̃)[1] #Innovation
    S = (C*Σ̃*C')[1] + V #Innovation Covariance

    L = Σ̃*C'*inv(S) #Kalman Gain

    x̂hist[:,k] = x̃ + L*z
    Σhist[:,:,k] = (I-L*C)*Σ̃*(I-L*C)' + L*V*L'

    #LQR Controller
    uhist[k] = -(K*x̂hist[:,k])[1]
    
    #Run this on the stochastic dynamics
    xhist[:,k+1] .= A*xhist[:,k] + B*uhist[k] + sqrt(W)*randn(n)
end

In [ ]:
plot(xhist[1,:])
plot(x̂hist[1,:])

In [ ]:
#Covariance and Kalman gain converge to steady-state values in the LTI case
#(just like LQR gain + cost-to-go Hessian)
plot(Σhist[1,1,1:N-1])
plot(Σhist[2,2,1:N-1])
plot(Σhist[1,2,1:N-1])